In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

# Đường dẫn tới các thư mục
dataset_dir = '../Input/Dataset/'
face_compare_dir = '../Input/Face-Compare/'
output_file = '../Output/result.xlsx'

# Hàm đọc ảnh từ folder
def read_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Đọc ảnh xám
        if img is not None:
            img_resized = cv2.resize(img, (100, 100))  # Resize về kích thước chuẩn
            images.append(img_resized.flatten())  # Chuyển ảnh thành vector
            labels.append(label)  # Gán nhãn là tên folder
    return images, labels

In [2]:
# Bước 1: Đọc và train dữ liệu từ Dataset
X_train = []
y_train = []

for person_folder in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, person_folder)
    if os.path.isdir(folder_path):
        images, labels = read_images_from_folder(folder_path, person_folder)
        X_train.extend(images)
        y_train.extend(labels)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [3]:
# Bước 2: Áp dụng PCA để giảm chiều
n_components = 100  # Số lượng eigenfaces
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)

In [4]:
# Bước 3: Đọc và xử lý ảnh từ Face-Compare
face_compare_images = []
face_compare_names = []

for filename in os.listdir(face_compare_dir):
    img_path = os.path.join(face_compare_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img_resized = cv2.resize(img, (100, 100))
        face_compare_images.append(img_resized.flatten())
        face_compare_names.append(filename)

face_compare_images = np.array(face_compare_images)
face_compare_scaled = scaler.transform(face_compare_images)
face_compare_pca = pca.transform(face_compare_scaled)

In [5]:
# Bước 4: So sánh các ảnh từ Face-Compare với các khuôn mặt đã train
results = []

for i, face_pca in enumerate(face_compare_pca):
    distances = euclidean_distances([face_pca], X_train_pca)
    min_distance_index = np.argmin(distances)

    if distances[0, min_distance_index] < 1000:  # Ngưỡng so sánh, có thể điều chỉnh
        matched_person = y_train[min_distance_index]
    else:
        matched_person = 'null'

    results.append([face_compare_names[i], matched_person])

In [6]:
# Bước 5: Xuất kết quả ra file Excel
df_results = pd.DataFrame(results, columns=['Image Name', 'Matched Person'])
df_results.to_excel(output_file, index=False)

print(f"Kết quả đã được lưu vào file {output_file}")

Kết quả đã được lưu vào file ../Output/result.xlsx
